# 0. Install and Import Dependencies

In [149]:
!pip install mediapipe opencv-python pandas scikit-learn

In [150]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv

In [151]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

# 1. Make Some Detections

In [152]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )

        # 2. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord(' '):
            break

cap.release()
cv2.destroyAllWindows()

# 2. Capture Landmarks & Export to CSV
<!--<img src="https://i.imgur.com/8bForKY.png">-->
<!--<img src="https://i.imgur.com/AzKNp7A.png">-->

In [153]:
import csv
import os
import numpy as np

In [154]:
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
num_coords

501

In [159]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [160]:
with open('facedata.csv', mode='a', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [165]:
class_name = "Sad"

In [166]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )

        # 2. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('facedata.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord(' '):
            break

cap.release()
cv2.destroyAllWindows()

# 3. Train Custom Model Using Scikit Learn

## 3.1 Read in Collected Data and Process

In [167]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [168]:
df = pd.read_csv('facedata.csv')

In [169]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Surprised,0.626545,0.577911,-0.546120,0.999995,0.642015,0.533612,-0.509707,0.999973,0.651752,...,-0.003802,0.0,0.673951,0.508112,0.012465,0.0,0.677387,0.501906,0.013212,0.0
1,Surprised,0.557906,0.570606,-0.460678,0.999973,0.574950,0.532545,-0.402876,0.999917,0.586311,...,-0.006361,0.0,0.598196,0.532841,0.004938,0.0,0.601718,0.526455,0.005265,0.0
2,Surprised,0.553265,0.570878,-0.439182,0.999960,0.567197,0.533274,-0.385459,0.999889,0.577298,...,-0.006342,0.0,0.591826,0.534553,0.004463,0.0,0.595320,0.528270,0.004841,0.0
3,Surprised,0.547327,0.568709,-0.462168,0.999948,0.561770,0.532593,-0.408931,0.999865,0.571110,...,-0.004664,0.0,0.586286,0.534029,0.005774,0.0,0.589791,0.528049,0.006047,0.0
4,Surprised,0.545834,0.568550,-0.492036,0.999939,0.559019,0.532361,-0.432491,0.999841,0.567008,...,-0.006333,0.0,0.583311,0.534683,0.003999,0.0,0.586877,0.528937,0.004242,0.0


In [170]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
499,Sad,0.535720,0.546731,-0.835718,0.999645,0.556084,0.501906,-0.771746,0.999463,0.566465,...,-0.012105,0.0,0.582461,0.518867,0.003274,0.0,0.586430,0.513897,0.003687,0.0
500,Sad,0.534370,0.546416,-0.822427,0.999651,0.554482,0.501773,-0.759020,0.999476,0.564989,...,-0.014121,0.0,0.580142,0.518790,-0.000594,0.0,0.584487,0.512621,-0.000260,0.0
501,Sad,0.533077,0.546996,-0.796829,0.999648,0.553186,0.501953,-0.734541,0.999470,0.563783,...,-0.012487,0.0,0.583829,0.518447,0.004661,0.0,0.587863,0.512662,0.005331,0.0
502,Sad,0.533069,0.548121,-0.789704,0.999594,0.553002,0.502758,-0.725130,0.999407,0.563432,...,-0.012646,0.0,0.581494,0.511537,0.003502,0.0,0.585667,0.505831,0.004094,0.0
503,Sad,0.533053,0.548276,-0.714295,0.999493,0.552701,0.502896,-0.648582,0.999281,0.562954,...,-0.012921,0.0,0.585234,0.502276,0.004690,0.0,0.589216,0.496395,0.005523,0.0


In [171]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [172]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [173]:
y_test

64     Surprised
305        Happy
398        Happy
123    Surprised
254        Happy
         ...    
382        Happy
108    Surprised
403          Sad
99     Surprised
102    Surprised
Name: class, Length: 152, dtype: object

## 3.2 Train Machine Learning Classification Model

In [174]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [175]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [176]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [177]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [178]:
fit_models['rc'].predict(X_test)

array(['Surprised', 'Happy', 'Happy', 'Surprised', 'Happy', 'Happy',
       'Surprised', 'Sad', 'Surprised', 'Happy', 'Surprised', 'Surprised',
       'Happy', 'Surprised', 'Sad', 'Happy', 'Sad', 'Surprised', 'Sad',
       'Surprised', 'Happy', 'Surprised', 'Sad', 'Surprised', 'Happy',
       'Surprised', 'Surprised', 'Happy', 'Happy', 'Happy', 'Surprised',
       'Surprised', 'Surprised', 'Happy', 'Happy', 'Sad', 'Happy',
       'Happy', 'Happy', 'Sad', 'Happy', 'Surprised', 'Happy', 'Happy',
       'Surprised', 'Happy', 'Happy', 'Happy', 'Surprised', 'Sad',
       'Surprised', 'Sad', 'Sad', 'Happy', 'Sad', 'Surprised', 'Happy',
       'Happy', 'Happy', 'Happy', 'Surprised', 'Happy', 'Surprised',
       'Happy', 'Sad', 'Sad', 'Sad', 'Happy', 'Happy', 'Happy', 'Happy',
       'Happy', 'Happy', 'Surprised', 'Happy', 'Surprised', 'Happy',
       'Happy', 'Happy', 'Happy', 'Sad', 'Surprised', 'Sad', 'Happy',
       'Sad', 'Surprised', 'Happy', 'Surprised', 'Surprised', 'Surprised',
      

## 3.3 Evaluate and Serialize Model 

In [179]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [180]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 0.9868421052631579
rc 1.0
rf 0.993421052631579
gb 0.9868421052631579


In [181]:
fit_models['rf'].predict(X_test)

array(['Surprised', 'Happy', 'Happy', 'Surprised', 'Happy', 'Happy',
       'Surprised', 'Sad', 'Surprised', 'Happy', 'Surprised', 'Surprised',
       'Happy', 'Surprised', 'Sad', 'Happy', 'Sad', 'Surprised', 'Sad',
       'Surprised', 'Happy', 'Surprised', 'Sad', 'Surprised', 'Happy',
       'Surprised', 'Surprised', 'Happy', 'Happy', 'Happy', 'Surprised',
       'Surprised', 'Surprised', 'Happy', 'Happy', 'Sad', 'Happy',
       'Happy', 'Happy', 'Sad', 'Happy', 'Surprised', 'Happy', 'Happy',
       'Surprised', 'Happy', 'Happy', 'Happy', 'Surprised', 'Sad',
       'Surprised', 'Sad', 'Sad', 'Happy', 'Sad', 'Surprised', 'Happy',
       'Happy', 'Happy', 'Happy', 'Surprised', 'Happy', 'Surprised',
       'Happy', 'Sad', 'Sad', 'Sad', 'Happy', 'Happy', 'Happy', 'Happy',
       'Happy', 'Happy', 'Surprised', 'Happy', 'Surprised', 'Happy',
       'Happy', 'Happy', 'Happy', 'Sad', 'Surprised', 'Sad', 'Happy',
       'Sad', 'Surprised', 'Happy', 'Surprised', 'Surprised', 'Surprised',
      

In [182]:
y_test

64     Surprised
305        Happy
398        Happy
123    Surprised
254        Happy
         ...    
382        Happy
108    Surprised
403          Sad
99     Surprised
102    Surprised
Name: class, Length: 152, dtype: object

In [183]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# 4. Make Detections with Model

In [186]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [187]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [188]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )

        # 2. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
#             # Append class name 
#             row.insert(0, class_name)
            
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row) 

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            #print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord(' '):
            break

cap.release()
cv2.destroyAllWindows()

C:\Users\Algor Lombako\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Algor Lombako\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Algor Lombako\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Algor Lombako\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Algor Lombako\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Algor Lombako\anaconda3\Lib\site-packages\sklearn\b